## Passo 1: Pré-processamento do texto

In [12]:
# Instala NTKL
!pip install nltk

# Baixa os recursos necessários usando o NLTK Downloader
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\victo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [13]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import string

# Função de pré-processamento do texto
def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()
    # Remover pontuações
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenização
    tokens = word_tokenize(text)
    # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Aplicar stemming
    stemmer = SnowballStemmer('portuguese')
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    # Reagrupar em uma string
    return ' '.join(stemmed_tokens)

# Carregar os dados
df = pd.read_excel('SECOES_IBGE_DESCRICAO_ATUALIZADA.xlsx')

# Aplicar a função de pré-processamento em cada descrição
df['Descrições_Processadas'] = df['Descrições'].apply(preprocess_text)

print(df[['Descrições', 'Descrições_Processadas']])


                                           Descrições  \
0   Iniciativas neste setor englobam a exploração ...   
1   O setor foca na extração de minerais brutos, f...   
2   Abrange a transformação de matérias-primas em ...   
3   Este setor fornece energia elétrica, gás natur...   
4   Engloba gestão de água, esgoto, resíduos e des...   
5   O setor de Construção abrange desde a edificaç...   
6   Essa seção inclui venda e manutenção de veícul...   
7   Este setor engloba o transporte de passageiros...   
8   Compreende hotéis, restaurantes e serviços de ...   
9   Engloba as indústrias de TI, telecomunicações ...   
10  Este setor contempla operações bancárias, de s...   
11  Engloba operações imobiliárias como locação e ...   
12  Abrange serviços especializados como consultor...   
13  Inclui serviços de apoio como recursos humanos...   
14  Este setor é responsável pela governança, serv...   
15  Inclui ensino de todos os níveis e modalidades...   
16  Este setor abarca o atendim

## Passo 2: Vetorização das Descrições

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inicializar o vetorizador
tfidf_vectorizer = TfidfVectorizer()

# Ajustar e transformar os dados
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Descrições_Processadas'])

## Passo 3

In [22]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular a matriz de similaridade do cosseno
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# A matriz de similaridade será uma matriz N x N, onde N é o número de descrições

## Passo 4

In [26]:
import numpy as np

# Supondo que 'cosine_sim_matrix' seja sua matriz de similaridade do cosseno
np.fill_diagonal(cosine_sim_matrix, 0)  # Passo 1
max_similarity = cosine_sim_matrix.max(axis=1)  # Passo 2

# Adicione a similaridade máxima ao DataFrame
df['Max_Similarity'] = max_similarity  # Passo 3

# Normalizar a variável de similaridade máxima, se necessário
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['Max_Similarity_Norm'] = scaler.fit_transform(df[['Max_Similarity']])  # Passo 4

# Verifique se as descrições com valores mais altos são as mais similares
print(df[['Descrições', 'Max_Similarity', 'Max_Similarity_Norm']].sort_values(by='Max_Similarity', ascending=False))  # Passo 5

# Agora 'Max_Similarity_Norm' está pronta para ser usada em algoritmos de clusterização

                                           Descrições  Max_Similarity  \
16  Este setor abarca o atendimento de saúde e ass...        0.259513   
14  Este setor é responsável pela governança, serv...        0.259513   
17  Engloba a gestão e promoção de atividades cult...        0.180835   
3   Este setor fornece energia elétrica, gás natur...        0.153149   
1   O setor foca na extração de minerais brutos, f...        0.153149   
15  Inclui ensino de todos os níveis e modalidades...        0.149749   
12  Abrange serviços especializados como consultor...        0.149328   
9   Engloba as indústrias de TI, telecomunicações ...        0.149328   
5   O setor de Construção abrange desde a edificaç...        0.148721   
4   Engloba gestão de água, esgoto, resíduos e des...        0.148705   
2   Abrange a transformação de matérias-primas em ...        0.139017   
19  Inclui serviços prestados por trabalhadores do...        0.138174   
8   Compreende hotéis, restaurantes e serviços de .

In [28]:
df.to_csv('SECOES_IBGE_DESCRICAO_ATUALIZADA_passo4.csv')

## Identificando descrições mais similares entre si

In [30]:
import numpy as np

# Sua matriz de similaridade do cosseno como um array NumPy
cosine_sim_matrix = np.array(cosine_sim_matrix.copy())  # Insira sua matriz aqui

# Número de descrições (linhas da matriz)
num_descriptions = cosine_sim_matrix.shape[0]

# Inicializar uma lista para armazenar os índices das descrições mais similares
most_similar_indices = []

# Iterar sobre cada descrição
for i in range(num_descriptions):
    # Ignorar a própria descrição zerando a similaridade com ela mesma
    cosine_sim_matrix[i, i] = 0
    
    # Encontrar o índice da descrição mais similar
    most_similar_idx = np.argmax(cosine_sim_matrix[i])
    
    # Armazenar o índice encontrado
    most_similar_indices.append(most_similar_idx)

# Restaurar a diagonal principal para 1 (se necessário posteriormente)
np.fill_diagonal(cosine_sim_matrix, 1)

# Exibir os pares de descrições mais similares
for i, idx in enumerate(most_similar_indices):
    print(f"Descrição {i} é mais similar à descrição {idx} com similaridade de {cosine_sim_matrix[i, idx]:.4f}")


Descrição 0 é mais similar à descrição 2 com similaridade de 0.1318
Descrição 1 é mais similar à descrição 3 com similaridade de 0.1531
Descrição 2 é mais similar à descrição 12 com similaridade de 0.1390
Descrição 3 é mais similar à descrição 1 com similaridade de 0.1531
Descrição 4 é mais similar à descrição 17 com similaridade de 0.1487
Descrição 5 é mais similar à descrição 3 com similaridade de 0.1487
Descrição 6 é mais similar à descrição 3 com similaridade de 0.1118
Descrição 7 é mais similar à descrição 14 com similaridade de 0.1069
Descrição 8 é mais similar à descrição 15 com similaridade de 0.1356
Descrição 9 é mais similar à descrição 12 com similaridade de 0.1493
Descrição 10 é mais similar à descrição 0 com similaridade de 0.1053
Descrição 11 é mais similar à descrição 4 com similaridade de 0.1198
Descrição 12 é mais similar à descrição 9 com similaridade de 0.1493
Descrição 13 é mais similar à descrição 0 com similaridade de 0.1196
Descrição 14 é mais similar à descrição

In [33]:
df

,SECOES_IBGE,Descrições,Descrições_Processadas,Max_Similarity,Max_Similarity_Norm
0,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",Iniciativas neste setor englobam a exploração ...,inic nest setor englob explor recurs natur pro...,0.131782,0.171593
1,INDÚSTRIAS EXTRATIVAS,"O setor foca na extração de minerais brutos, f...",setor foc extraçã miner brut fundament econom ...,0.153149,0.310173
2,INDÚSTRIAS DE TRANSFORMAÇÃO,Abrange a transformação de matérias-primas em ...,abrang transform matériasprim produt acab esse...,0.139017,0.218516
3,ELETRICIDADE E GÁS,"Este setor fornece energia elétrica, gás natur...",setor fornec energ elétr gás natural águ quent...,0.153149,0.310173
4,"ÁGUA, ESGOTO, ATIVIDADES DE GESTÃO DE RESÍDUOS...","Engloba gestão de água, esgoto, resíduos e des...",englob gestã águ esgot resídu descontamin enfr...,0.148705,0.281348
5,CONSTRUÇÃO,O setor de Construção abrange desde a edificaç...,setor construçã abrang desd edific estrutur re...,0.148721,0.281454
6,COMÉRCIO; REPARAÇÃO DE VEÍCULOS AUTOMOTORES E ...,Essa seção inclui venda e manutenção de veícul...,seçã inclu vend manutençã veícul motociclet en...,0.111793,0.041955
7,"TRANSPORTE, ARMAZENAGEM E CORREIO",Este setor engloba o transporte de passageiros...,setor englob transport passageir carg servic l...,0.106877,0.010070
8,ALOJAMENTO E ALIMENTAÇÃO,"Compreende hotéis, restaurantes e serviços de ...",compreend hot restaur servic catering enfrent ...,0.135643,0.196631
9,INFORMAÇÃO E COMUNICAÇÃO,"Engloba as indústrias de TI, telecomunicações ...",englob indústr ti telecomunic míd prioriz segu...,0.149328,0.285392


In [34]:
df.to_excel('SECOES_IBGE_DESCRICAO_ATUALIZADA_output.xlsx')